In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Using Vertex AI Matching Engine for StackOverflow Questions

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/matching_engine/sdk_matching_engine_create_stack_overflow_embeddings.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/matching_engine/sdk_matching_engine_create_stack_overflow_embeddings.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
      <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/matching_engine/sdk_matching_engine_create_stack_overflow_embeddings.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>
</table>

## Overview

This example demonstrates how to create text-to-image embeddings using the StackOverflow dataset and the sentence-T5 model. These are uploaded to Vertex AI Matching Engine service. It is a high scale, low latency solution, to find similar vectors (or more specifically "embeddings") for a large corpus. Moreover, it is a fully managed offering, further reducing operational overhead. It is built upon [Approximate Nearest Neighbor (ANN) technology](https://ai.googleblog.com/2020/07/announcing-scann-efficient-vector.html) developed by Google Research.

**Pre-requisite**: This notebook requires you to already have a VPC network set up. See the "Prepare a VPC network" section in [Create Vertex AI Matching Engine index notebook](https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/matching_engine/sdk_matching_engine_for_indexing.ipynb).

Learn more about [Vertex AI Matching Engine](https://cloud.google.com/vertex-ai/docs/matching-engine/overview).

### Objective

In this notebook, you learn how to encode custom text embeddings, create an  Approximate Nearest Neighbor (ANN) Index, and query against indexes.

This tutorial uses the following Google Cloud ML services:

- `Vertex AI Matching Engine`

The steps performed include:

* Create ANN Index
* Create an IndexEndpoint with VPC Network
* Deploy ANN Index
* Perform online query


### Dataset

The dataset used for this tutorial is the [StackOverflow dataset](https://console.cloud.google.com/marketplace/product/stack-exchange/stack-overflow).

> Stack Overflow is the largest online community for programmers to learn, share their knowledge, and advance their careers. Updated on a quarterly basis, this BigQuery dataset includes an archive of Stack Overflow content, including posts, votes, tags, and badges. This dataset is updated to mirror the Stack Overflow content on the Internet Archive, and is also available through the Stack Exchange Data Explorer.

## Installation

Install the latest version of Cloud Storage, BigQuery and Vertex AI SDKs for Python.

In [ ]:
# Install the packages
! pip3 install --upgrade google-cloud-aiplatform \
                        google-cloud-storage --upgrade

### Colab only: Uncomment the following cell to restart the kernel.

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

## Before you begin
#### Set your project ID

**If you don't know your project ID**, try the following:
* Run `gcloud config list`.
* Run `gcloud projects list`.
* See the support page: [Locate the project ID](https://support.google.com/googleapi/answer/7014113)

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

#### Region

You can also change the `REGION` variable used by Vertex AI. Learn more about [Vertex AI regions](https://cloud.google.com/vertex-ai/docs/general/locations).

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

### Authenticate your Google Cloud account

Depending on your Jupyter environment, you may have to manually authenticate. Follow the relevant instructions below.

**1. Vertex AI Workbench**
* Do nothing as you are already authenticated.

**2. Local JupyterLab instance, uncomment and run:**

In [ ]:
# ! gcloud auth login

**3. Colab, uncomment and run:**

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

**4. Service account or other**
* See how to grant Cloud Storage permissions to your service account at https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples.

* Authentication: Rerun the `gcloud auth login` command in the Vertex AI Workbench notebook terminal when you are logged out and need the credential again.

### Colab only: Uncomment the following cell to restart the kernel.

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

### Create a Cloud Storage bucket

Create a storage bucket to store intermediate artifacts such as datasets.

In [ ]:
BUCKET_URI = "gs://your-bucket-name-unique"  # @param {type:"string"}

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [ ]:
! gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_URI

## Prepare the data

You will use [Stack Overflow dataset](https://console.cloud.google.com/marketplace/product/stack-exchange/stack-overflow) of question and answers hosted on BigQuery.

> This public dataset is hosted in Google BigQuery and is included in BigQuery's 1TB/mo of free tier processing. This means that each user receives 1TB of free BigQuery processing every month, which can be used to run queries on this public dataset.

In [ ]:
%%time
from google.cloud import bigquery

client = bigquery.Client(project=PROJECT_ID)

NUM_ROWS = 1000

QUERY = f"""
        SELECT distinct q.id, q.title, q.body, q.tags, a.body as answers, a.score 
        FROM (SELECT * FROM `bigquery-public-data.stackoverflow.posts_questions` where Score>0 ORDER BY View_Count desc) AS q 
        INNER JOIN (SELECT * FROM `bigquery-public-data.stackoverflow.posts_answers`  where Score>0 ORDER BY Score desc) AS a ON q.id = a.parent_id 
        where q.tags like '%python%'
        LIMIT {NUM_ROWS};
        """

query_job = client.query(QUERY)
rows = query_job.result()

In [ ]:
# Convert to a dataframe
df = rows.to_dataframe()

# Examine the data
df.head()

In [ ]:
# Extract the question ids and question text
ids = df.id.tolist()
questions = df.title.tolist()

# Verify the length
len(ids)

#### Instantiate the text encoding model

Use the [sentence-t5 encoder](https://tfhub.dev/google/sentence-t5/st5-base/1) developed by Google for converting text to embeddings.

> The sentence-T5 family of models encode text into high-dimensional vectors that can be used for text classification, semantic similarity, clustering and other natural language processing tasks.
>
> Our model is built on top of T5 (i.e. the Text-To-Text Transfer Transformer). It is trained on a variety of data sources and initialized from pre-trained T5 models with different model sizes as described in [1]. The input is variable-length English text and the output is a 768-dimensional vector. The sentence-T5 base model employs a 12-layer transformer architecture as the T5 base model does.

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
# Registers the ops.
import tensorflow_text as text  # noqa: F401

hub_url = "https://tfhub.dev/google/sentence-t5/st5-base/1"

encoder = hub.KerasLayer(hub_url)

#### Defining an encoding function

Define a function to be used later that will take sentences and convert them to embeddings.

In [ ]:
from typing import List

import numpy as np

NUM_SENTENCES_IN_CHUNK = 100


def encode_text_to_embedding(
    text_encoder: hub.KerasLayer, sentences: List[str]
) -> np.ndarray:
    embeddings_list = []

    # The encoding models in TF hub have trouble processing too many strings at once, so we process them in chunks.
    sentence_chunks = np.array_split(
        sentences, (len(sentences) // NUM_SENTENCES_IN_CHUNK) + 1
    )
    for index, chunk in enumerate(sentence_chunks):
        print(f"\tProcessing {(index+1)/len(sentence_chunks)*100:.1f}")
        embeddings_list.append(text_encoder(tf.constant(chunk)))

    return np.squeeze(np.column_stack(embeddings_list))

#### Test the encoding function

Encode a subset of data and see if the embeddings and distance metrics make sense.

According to (sentence-T5 research paper)[https://arxiv.org/pdf/2108.08877.pdf], the similarity of embeddings is calculated using the dot-product. 

In [ ]:
# Encode 500 questions
questions = df.title.tolist()[:500]
question_embeddings = convert_text_to_embeddings(
    text_encoder=encoder, sentences=questions
)

In [ ]:
question_index = 10

print(f"Query question = {questions[question_index]}")
scores = np.dot(question_embeddings[question_index], question_embeddings)

# Print top 20 matches
for index, (question, score) in enumerate(
    sorted(zip(questions, scores), key=lambda x: x[1], reverse=True)[:20]
):
    print(f"\t{index}: {question}: {score}")

#### Save the train split in JSONL format.

The data must be formatted in JSONL format, which means each embedding dictionary is written as a JSON string on its own line.

Additionally, to demonstrate the filtering functionality, the `restricts` key is set such that each embedding has a different `class`, `even` or `odd`. These are used during the later matching step to filter for results.
See additional information of filtering here: https://cloud.google.com/vertex-ai/docs/matching-engine/filtering

In [ ]:
import json
import tempfile

NUM_ELEMENTS_IN_CHUNK = 1000
embeddings_file = tempfile.NamedTemporaryFile(suffix=".json", delete=False)
id_question_chunks = (
    list(zip(ids, questions))[i : i + NUM_ELEMENTS_IN_CHUNK]
    for i in range(0, len(questions), NUM_ELEMENTS_IN_CHUNK)
)

with open(embeddings_file.name, "a") as f:
    progress = 0
    for chunk in id_question_chunks:
        id_chunk, question_chunk = zip(*chunk)

        progress += 1
        print(f"Processing chunk {progress}/{len(questions)//NUM_ELEMENTS_IN_CHUNK}")

        question_chunk_embeddings = encode_text_to_embedding(
            text_encoder=encoder, sentences=question_chunk
        )

        # Append to file
        embeddings_formatted = [
            json.dumps(
                {
                    "id": str(id),
                    "embedding": [str(value) for value in embedding],
                }
            )
            + "\n"
            for id, embedding in zip(id_chunk, question_chunk_embeddings)
        ]
        f.writelines(embeddings_formattedrmatted)

Upload the training data to GCS.

In [ ]:
EMBEDDINGS_INITIAL_URI = f"{BUCKET_URI}/matching_engine/initial/embeddings_unique/"
! gsutil cp {embeddings_file.name} {EMBEDDINGS_INITIAL_URI}

## Create Indexes


### Create ANN Index (for Production Usage)

In [ ]:
DIMENSIONS = len(question_embeddings[0])
DISPLAY_NAME = "stack_overflow"
DESCRIPTION = "questions from stackoverflow"

Create the ANN index configuration:

To learn more about configuring the index, see [Input data format and structure](https://cloud.google.com/vertex-ai/docs/matching-engine/match-eng-setup#input-data-format).


In [ ]:
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_URI)

In [ ]:
tree_ah_index = aiplatform.MatchingEngineIndex.create_tree_ah_index(
    display_name=DISPLAY_NAME,
    contents_delta_uri=EMBEDDINGS_INITIAL_URI,
    dimensions=DIMENSIONS,
    approximate_neighbors_count=150,
    distance_measure_type="DOT_PRODUCT_DISTANCE",
    leaf_node_embedding_count=500,
    leaf_nodes_to_search_percent=7,
    description=DESCRIPTION,
)

In [ ]:
INDEX_RESOURCE_NAME = tree_ah_index.resource_name
INDEX_RESOURCE_NAME

Using the resource name, you can retrieve an existing MatchingEngineIndex.

In [ ]:
tree_ah_index = aiplatform.MatchingEngineIndex(index_name=INDEX_RESOURCE_NAME)

## Create an IndexEndpoint with VPC Network

In [ ]:
# Retrieve the project number
PROJECT_NUMBER = !gcloud projects list --filter="PROJECT_ID:'{PROJECT_ID}'" --format='value(PROJECT_NUMBER)'
PROJECT_NUMBER = PROJECT_NUMBER[0]

VPC_NETWORK = "[your-network-name]"
VPC_NETWORK_FULL = "projects/{}/global/networks/{}".format(PROJECT_NUMBER, VPC_NETWORK)
VPC_NETWORK_FULL

In [ ]:
my_index_endpoint = aiplatform.MatchingEngineIndexEndpoint.create(
    display_name=DISPLAY_NAME,
    description=DISPLAY_NAME,
    network=VPC_NETWORK_FULL,
)

## Deploy Indexes

### Deploy ANN Index

In [ ]:
DEPLOYED_INDEX_ID = "deployed_index_id_unique"

In [ ]:
my_index_endpoint = my_index_endpoint.deploy_index(
    index=tree_ah_index, deployed_index_id=DEPLOYED_INDEX_ID
)

my_index_endpoint.deployed_indexes

## Create Online Queries

After you built your indexes, you may query against the deployed index to find nearest neighbours.

In [ ]:
test_embeddings = encode_text_to_embedding(
    text_encoder=encoder, sentences=["How do I install tensorflow with GPU support?"]
)

In [ ]:
# Test query
response = my_index_endpoint.match(
    deployed_index_id=DEPLOYED_INDEX_ID,
    queries=test_embeddings,
    num_neighbors=NUM_NEIGHBOURS,
)

response

## Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.
You can also manually delete resources that you created by running the following code.

In [ ]:
# Force undeployment of indexes and delete endpoint
my_index_endpoint.delete(force=True)

In [ ]:
# Delete indexes
tree_ah_index.delete()